# 3D Ball

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch

from mlagents_envs.environment import UnityEnvironment
from gym_unity.envs import UnityToGymWrapper
import numpy as np

from agent import Agent
from ddpg_learning import ddpg

In [2]:
torch.cuda.is_available()

True

In [3]:
# Initialize the Environment
unity_env = UnityEnvironment(file_name="Visual3DBall\\UnityEnvironment.app")
env = UnityToGymWrapper(unity_env)

# Get the action size
action_size = 2

# Get the state size
state_shape = (84,84,12)

# Get number of agents
num_agents = 1


2021-04-19 11:48:59 INFO [environment.py:113] Connected to Unity environment with package version 1.9.0-preview and communication version 1.5.0
2021-04-19 11:49:00 INFO [environment.py:282] Connected new brain:
Visual3DBall?team=0
C:\Users\me\anaconda3\envs\unity\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [5]:
#Initialize the Agent with given hyperparameters

BUFFER_SIZE = int(3e4)  # replay buffer size
BATCH_SIZE = 128       # batch size
GAMMA = 0.99            # discount factor
TAU = 1e-2              # for soft update of target parameters 1e-4
LR_ACTOR = 5e-4         # learning rate of the actor 1e-4
LR_CRITIC = 5e-3        # learning rate of the critic 1e-4
UPDATE_EVERY = 1        # how often to update the network

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #Torch device to use

agent = Agent(state_shape=state_shape,
              action_size=action_size,
              num_agents=num_agents,
              buffer_size=BUFFER_SIZE,
              batch_size=BATCH_SIZE,
              gamma=GAMMA,
              tau=TAU,
              learning_rate_actor=LR_ACTOR,
              learning_rate_critic=LR_CRITIC,
              device=device,
              update_every=UPDATE_EVERY,
              random_seed=42)


agent.actor_local.load_state_dict(torch.load('checkpoints\\prioritized_replay\\checkpoint_actor_local_3000.pth'))
agent.critic_local.load_state_dict(torch.load('checkpoints\\prioritized_replay\\checkpoint_critic_local_3000.pth'))
agent.actor_target.load_state_dict(torch.load('checkpoints\\prioritized_replay\\checkpoint_actor_target_3000.pth'))
agent.critic_target.load_state_dict(torch.load('checkpoints\\prioritized_replay\\checkpoint_critic_target_3000.pth'))

<All keys matched successfully>

In [ ]:
# Train the agent

AVERAGE_SCORE_SOLVED=100

scores, num_episodes_solved = ddpg(env=env,
                                   agent=agent,
                                   num_agents=num_agents,
                                   average_score_solved=AVERAGE_SCORE_SOLVED)


Episode 100	Average Score: 1.04	Max: 3.20	Max Timesteps: 43     
Elapsed Time: 3 minutes 31 seconds	Total Timesteps: 2137
Episode 126	Average Score: 0.94	Score: 0.50	Timesteps: 16               

In [ ]:
#prioritized replay

BUFFER_SIZE = int(3e4)  # replay buffer size
BATCH_SIZE = 128        # batch size
GAMMA = 0.99            # discount factor
TAU = 1e-2              # for soft update of target parameters
LR_ACTOR = 5e-4         # learning rate of the actor
LR_CRITIC = 5e-3        # learning rate of the critic
UPDATE_EVERY = 1        # how often to update the network


Episode 100	Average Score: 0.42	Max: 1.60	Max Timesteps: 27     
Elapsed Time: 2 minutes 28 seconds	Total Timesteps: 1519
Episode 200	Average Score: 0.62	Max: 1.90	Max Timesteps: 30     
Elapsed Time: 5 minutes 18 seconds	Total Timesteps: 3235
Episode 300	Average Score: 0.66	Max: 3.30	Max Timesteps: 44     
Elapsed Time: 8 minutes 12 seconds	Total Timesteps: 4998
Episode 400	Average Score: 1.37	Max: 3.40	Max Timesteps: 45     
Elapsed Time: 12 minutes 18 seconds	Total Timesteps: 7472
Episode 500	Average Score: 1.43	Max: 4.20	Max Timesteps: 53     
Elapsed Time: 16 minutes 29 seconds	Total Timesteps: 10000
Episode 600	Average Score: 1.43	Max: 4.50	Max Timesteps: 56     
Elapsed Time: 20 minutes 39 seconds	Total Timesteps: 12525
Episode 700	Average Score: 1.57	Max: 4.50	Max Timesteps: 56     
Elapsed Time: 25 minutes 4 seconds	Total Timesteps: 15192
Episode 800	Average Score: 1.57	Max: 4.90	Max Timesteps: 60     
Elapsed Time: 29 minutes 29 seconds	Total Timesteps: 17866
Episode 900	Average Score: 1.39	Max: 4.10	Max Timesteps: 52     
Elapsed Time: 33 minutes 36 seconds	Total Timesteps: 20356
Episode 1000	Average Score: 1.66	Max: 6.10	Max Timesteps: 72     
Elapsed Time: 38 minutes 10 seconds	Total Timesteps: 23113
Episode 1100	Average Score: 1.58	Max: 4.20	Max Timesteps: 53     
Elapsed Time: 42 minutes 37 seconds	Total Timesteps: 25794
Episode 1200	Average Score: 1.45	Max: 4.20	Max Timesteps: 53     
Elapsed Time: 46 minutes 50 seconds	Total Timesteps: 28346
Episode 1300	Average Score: 1.66	Max: 7.40	Max Timesteps: 85     
Elapsed Time: 51 minutes 24 seconds	Total Timesteps: 31104
Episode 1400	Average Score: 1.77	Max: 6.90	Max Timesteps: 80     
Elapsed Time: 56 minutes 8 seconds	Total Timesteps: 33976
Episode 1500	Average Score: 1.76	Max: 5.30	Max Timesteps: 64     
Elapsed Time: 60 minutes 53 seconds	Total Timesteps: 36836
Episode 1600	Average Score: 1.83	Max: 6.40	Max Timesteps: 75     
Elapsed Time: 65 minutes 43 seconds	Total Timesteps: 39762
Episode 1700	Average Score: 1.76	Max: 7.60	Max Timesteps: 87     
Elapsed Time: 70 minutes 28 seconds	Total Timesteps: 42626
Episode 1800	Average Score: 1.55	Max: 5.20	Max Timesteps: 63     
Elapsed Time: 74 minutes 51 seconds	Total Timesteps: 45280
Episode 1900	Average Score: 1.74	Max: 7.90	Max Timesteps: 90     
Elapsed Time: 79 minutes 35 seconds	Total Timesteps: 48120
Episode 2000	Average Score: 1.66	Max: 5.90	Max Timesteps: 70     
Elapsed Time: 84 minutes 9 seconds	Total Timesteps: 50884
Episode 2100	Average Score: 1.79	Max: 8.60	Max Timesteps: 97     
Elapsed Time: 88 minutes 56 seconds	Total Timesteps: 53770
Episode 2200	Average Score: 1.63	Max: 7.60	Max Timesteps: 87     
Elapsed Time: 93 minutes 27 seconds	Total Timesteps: 56495
Episode 2300	Average Score: 1.82	Max: 5.20	Max Timesteps: 63     
Elapsed Time: 98 minutes 17 seconds	Total Timesteps: 59419
Episode 2400	Average Score: 1.67	Max: 5.10	Max Timesteps: 62     
Elapsed Time: 102 minutes 52 seconds	Total Timesteps: 62186
Episode 2500	Average Score: 1.97	Max: 8.20	Max Timesteps: 93     
Elapsed Time: 107 minutes 57 seconds	Total Timesteps: 65258
Episode 2600	Average Score: 1.98	Max: 6.70	Max Timesteps: 78     
Elapsed Time: 113 minutes 3 seconds	Total Timesteps: 68341
Episode 2700	Average Score: 1.88	Max: 8.30	Max Timesteps: 94     
Elapsed Time: 117 minutes 59 seconds	Total Timesteps: 71320
Episode 2800	Average Score: 1.73	Max: 6.00	Max Timesteps: 71     
Elapsed Time: 122 minutes 40 seconds	Total Timesteps: 74154
Episode 2900	Average Score: 1.96	Max: 7.30	Max Timesteps: 84     
Elapsed Time: 127 minutes 45 seconds	Total Timesteps: 77215
Episode 3000	Average Score: 1.87	Max: 10.80	Max Timesteps: 119     
Elapsed Time: 132 minutes 53 seconds	Total Timesteps: 80185


In [ ]:
#prioritized replay
#gradient clipping

BUFFER_SIZE = int(3e4)  # replay buffer size
BATCH_SIZE = 128        # batch size
GAMMA = 0.99            # discount factor
TAU = 1e-2              # for soft update of target parameters
LR_ACTOR = 5e-4         # learning rate of the actor
LR_CRITIC = 5e-3        # learning rate of the critic
UPDATE_EVERY = 1        # how often to update the network



In [ ]:
#Plot the training session (scores per episode averaged across all agents)

def plot_scores(scores):
    plt.plot(scores, color='royalblue')
    plt.title('Scores per episode')
    plt.ylabel('score')
    plt.xlabel('episode #')
    plt.show()
    
plot_scores(scores)          

In [ ]:
#See the trained agent in action.

env_info = env.reset(train_mode=False)[brain_name] # reset the environment
states = env_info.vector_observations              # get the current state

#Uncomment these lines to use a saved checkpoint:
#agent = Agent(state_size=state_size,
#              action_size=action_size,
#              num_agents=num_agents,
#              buffer_size=BUFFER_SIZE,
#              batch_size=BATCH_SIZE,
#              gamma=GAMMA,
#              tau=TAU,
#              learning_rate_actor=LR_ACTOR,
#              learning_rate_critic=LR_CRITIC,
#              device=device,
#              update_every=UPDATE_EVERY,
#              random_seed=42)
#agent.actor_local.load_state_dict(torch.load('solved_checkpoint_actor.pth'))

scores = np.zeros(num_agents) 
while True:
    actions = agent.act(states, add_noise=False)   # select an action
    env_info = env.step(actions)[brain_name]       # send the action to the environment
    next_states = env_info.vector_observations     # get the next state
    rewards = env_info.rewards                     # get the reward
    dones = env_info.local_done                    # see if episode has finished
    states = next_states                           # roll over the state to next time step
    scores += rewards                              # update the score
    if np.any(dones):                              # exit loop if episode finished
        break
    
print("Average Score: {}".format(np.mean(scores)))


In [ ]:
env.close()

In [ ]:
#a = env._get_vis_obs_shape()
#obs = env.step(action)
#obs[0][:,:,3:6].shape
#show_image(obs[0][:,:,0:3])
#show_image(obs[0][:,:,3:6])
#show_image(obs[0][:,:,6:9])
#show_image(obs[0][:,:,9:12])

In [ ]:
#See the trained agent in action.

for i in range(10):
    state = env.reset()
    scores = np.zeros(num_agents) 
    while True:
        actions = np.clip(np.random.normal(size=(2,)), -1, 1)
        next_state, reward, done, _ = env.step(actions)       # send the action to the environment
        scores += reward                              # update the score
        if done:                              # exit loop if episode finished
            break
    
    print("Average Score: {}".format(np.mean(scores)))


In [ ]:

def show_image(img):
    def scale_lumininance(img):
        return np.dot(img[...,:3], [0.299, 0.587, 0.114])

    def normalize(img):
        return img / 255

    img = scale_lumininance(img)
    img = normalize(img)

    plt.figure(figsize=(12,8))
    plt.imshow(img, cmap=plt.get_cmap('gray'))
    plt.axis('off')
    plt.show()

